In [1]:
%load_ext autoreload
%autoreload 2
import telepot
bot = telepot.Bot('436447719:AAGq_iK2hE3cPFhmL9Jh53g9HyVkw3SXZbg')
def qq(txt):
    bot.sendMessage(1114926, txt)

In [2]:
qq('start')

In [65]:
import json
import gzip
from itertools import islice
from collections import Counter, defaultdict
from operator import itemgetter
import pyarrow.parquet as pq
import pyarrow as pa

import pandas as pd
from tqdm.notebook import tqdm
import dill
import os
from pqdm.processes import pqdm
import re

from workers import *
    
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec

import random
import collections
import datetime
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [37]:
df = pd.read_parquet('df_tarin.pq')

In [38]:
df.columns

Index(['vacancy_id', 'name', 'description', 'area_id', 'creation_date',
       'work_schedule', 'work_experience', 'compensation_from',
       'compensation_to', 'currency', 'key_skills', 'employer',
       'specializations', 'is_test', 'part', 'employment', 'spec_len',
       'groups_of_spec', 'len_groups', 'html_tags', 'clear_text',
       'processed_text'],
      dtype='object')

In [39]:
pd.set_option('display.max_columns', 61)

In [40]:
df.head(2)

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,key_skills,employer,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags,clear_text,processed_text
4,2700005,Разработчик Python/django,<p>В небольшую дружную команду требуется опытн...,1,2019-09-03,fullDay,between3And6,70000.0,80000.0,RUR,[],845fe6266ab1,"[137, 221, 296]",False,10,full,3.0,[1],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, ul,...",В небольшую дружную команду требуется опытный ...,небольшой дружный команда требоваться опытный ...
5,2700006,Торговый представитель,<p>Обязанности: В компанию дистрибьютор продук...,53,2019-10-01,fullDay,noExperience,30000.0,60000.0,RUR,"[работа с текущей базой клиентов, Мерчендайзин...",23ead4a8f164,"[149, 242]",False,10,full,2.0,[17],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, p, ...",Обязанности: В компанию дистрибьютор продуктов...,обязанность компания дистрибьютор продукт пита...


In [41]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

d1 = datetime.date.today()

In [42]:
df['len_text'] = df['description'].str.len()
df['len_name'] = df['name'].str.len()

In [43]:
df.head(2)

,vacancy_id,name,description,area_id,creation_date,work_schedule,work_experience,compensation_from,compensation_to,currency,key_skills,employer,specializations,is_test,part,employment,spec_len,groups_of_spec,len_groups,html_tags,clear_text,processed_text,len_text,len_name
4,2700005,Разработчик Python/django,<p>В небольшую дружную команду требуется опытн...,1,2019-09-03,fullDay,between3And6,70000.0,80000.0,RUR,[],845fe6266ab1,"[137, 221, 296]",False,10,full,3.0,[1],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, ul,...",В небольшую дружную команду требуется опытный ...,небольшой дружный команда требоваться опытный ...,1105,25
5,2700006,Торговый представитель,<p>Обязанности: В компанию дистрибьютор продук...,53,2019-10-01,fullDay,noExperience,30000.0,60000.0,RUR,"[работа с текущей базой клиентов, Мерчендайзин...",23ead4a8f164,"[149, 242]",False,10,full,2.0,[17],1.0,"[p, /p, p, /p, p, /p, p, /p, p, /p, p, /p, p, ...",Обязанности: В компанию дистрибьютор продуктов...,обязанность компания дистрибьютор продукт пита...,583,22


In [44]:
df['num_words'] = df['processed_text'].apply(lambda x: len(x.split(' ')))
df['age'] = df['creation_date'].apply(lambda x: diff_month(d1,x))
df['len_ks'] = df['key_skills'].apply(len)
df['len_html'] = df['html_tags'].apply(len)

In [45]:
drop_cols = ['name', 'description', 'creation_date', 'key_skills', 'specializations', 
             'is_test', 'part', 'groups_of_spec', 'len_groups', 'html_tags', 'clear_text', 'processed_text']
cat_cols = ['work_schedule', 'work_experience', 'currency', 'employer', 'employment']

In [46]:
df.drop(columns=drop_cols, inplace = True)

In [47]:
df.columns

Index(['vacancy_id', 'area_id', 'work_schedule', 'work_experience',
       'compensation_from', 'compensation_to', 'currency', 'employer',
       'employment', 'spec_len', 'len_text', 'len_name', 'num_words', 'age',
       'len_ks', 'len_html'],
      dtype='object')

In [48]:
df.head(2)

,vacancy_id,area_id,work_schedule,work_experience,compensation_from,compensation_to,currency,employer,employment,spec_len,len_text,len_name,num_words,age,len_ks,len_html
4,2700005,1,fullDay,between3And6,70000.0,80000.0,RUR,845fe6266ab1,full,3.0,1105,25,90,10,0,50
5,2700006,53,fullDay,noExperience,30000.0,60000.0,RUR,23ead4a8f164,full,2.0,583,22,47,9,5,18


In [51]:
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].to_list())

In [85]:
df.head(2)

,vacancy_id,area_id,work_schedule,work_experience,compensation_from,compensation_to,currency,employer,employment,spec_len,len_text,len_name,num_words,age,len_ks,len_html
4,2700005,1,2,1,70000.0,80000.0,6,132398,0,3,1105,25,90,10,0,50
5,2700006,53,2,3,30000.0,60000.0,6,35980,0,2,583,22,47,9,5,18


In [116]:
model = xgb.XGBRegressor(n_estimators=200, max_depth = 10, objective ='reg:squarederror', learning_rate = 0.1, alpha=10, n_jobs=10)

In [84]:
df['spec_len'] = df['spec_len'].astype(int)

In [100]:
validation = dill.load(open('validatin.bin', 'rb'))

In [102]:
validation = set(validation)

In [103]:
df['is_vailidation'] = df['vacancy_id'].apply(lambda x: x in validation)

In [109]:
df_train = df[~df['is_vailidation']]
df_val = df[df['is_vailidation']]

In [110]:
print(df_train.shape, df_val.shape)

(1172160, 17) (284165, 17)


In [117]:
model.fit(df_train.drop(columns=['vacancy_id', 'spec_len', 'is_vailidation']), df_train['spec_len'])

XGBRegressor(alpha=10, base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=10, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [118]:
y_pred = model.predict(df_val.drop(columns=['vacancy_id', 'spec_len', 'is_vailidation']))

In [119]:
res = mean_squared_error(df_val['spec_len'], np.round(y_pred, 0))
qq(res)
print(res)

2.7812679253250754


In [120]:
y_pred = model.predict(df_train.drop(columns=['vacancy_id', 'spec_len', 'is_vailidation']))
res = mean_squared_error(df_train['spec_len'], np.round(y_pred, 0))
qq(res)
print(res)

2.5922169328419327


In [121]:
model2 = xgb.XGBClassifier(n_estimators=200, max_depth = 10, objective ='multi:softmax', learning_rate = 0.1, alpha=10, n_jobs=10)

In [122]:
model2.fit(df.drop(columns=['vacancy_id', 'spec_len']), df['spec_len'])

XGBClassifier(alpha=10, base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=10, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=10,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [123]:
y_pred2 = model2.predict(df.drop(columns=['vacancy_id', 'spec_len']))

In [124]:
mean_squared_error(df['spec_len'], y_pred2)

5.210587609221843